In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
from sklearn.model_selection import train_test_split,KFold,cross_val_score
from sklearn.svm import SVR
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from hyperopt import fmin,tpe,hp,Trials
from hyperopt.pyll.stochastic import sample
import random
import tensorflow as tf
import levenberg_marquardt as lm
import warnings
from keras.optimizers import Adam,SGD,RMSprop,Adadelta,Adagrad,Adamax,Nadam,Ftrl

LeakyReLU=tf.keras.layers.LeakyReLU(alpha=0.1)
warnings.filterwarnings("ignore")

random_state=87

In [2]:
cuo_water_data=pd.read_excel("Mech Project.xlsx").astype("float64")
cuo_water_data.head()

,Temperature,Nanparticle Heat Capacity,Volume fraction,CuO/water heat capacity
0,298.31,0.536,0.4,4.082
1,300.32,0.535,0.4,4.081
2,302.35,0.533,0.4,4.079
3,304.36,0.538,0.4,4.082
4,306.37,0.548,0.4,4.085


In [3]:
X=cuo_water_data[cuo_water_data.columns[0:3]]
y=cuo_water_data[cuo_water_data.columns[3]].values

In [4]:
y=y.reshape(-1,1)

In [5]:
sc_X=preprocessing.MinMaxScaler()
sc_y=preprocessing.MinMaxScaler()
X=sc_X.fit_transform(X)
y=sc_y.fit_transform(y)

In [6]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=random_state)

In [7]:
ann_params_space={
    'activation':hp.choice('activation',[0,1,2,3,4,5,6,7,8,9]),
    'optimizer':hp.choice('optimizer',[0,1,2,3,4,5,6,7]),
    'num_hidden_layers':hp.quniform('num_hidden_layers',1,6,1),
    'nodes':hp.quniform('nodes',1,200,1),
    'dropout_prob':hp.uniform('dropout_prob',0,1),
    'learning_rate': hp.loguniform('learning_rate',np.log(0.001),np.log(0.2)), 
    'batch_size':hp.quniform('batch_size',1, 100,1),
    'epochs':hp.quniform('epochs',10,500,10)
}

In [8]:
def objective_ann(params,X_train=X_train,y_train=y_train):
    learning_rate=params['learning_rate']
    
    optimizerL = ['SGD', 'Adam', 'RMSprop', 'Adadelta',
                'Adagrad', 'Adamax', 'Nadam', 'Ftrl']
    
    optimizerD = {'Adam': Adam(lr=learning_rate), 'SGD': SGD(lr=learning_rate),
                'RMSprop': RMSprop(lr=learning_rate), 'Adadelta': Adadelta(lr=learning_rate),
                'Adagrad': Adagrad(lr=learning_rate), 'Adamax': Adamax(lr=learning_rate),
                'Nadam': Nadam(lr=learning_rate), 'Ftrl': Ftrl(lr=learning_rate)}
    
    activationL = ['relu', 'sigmoid', 'softplus', 'softsign', 'tanh', 'selu',
                'elu', 'exponential', LeakyReLU, 'relu']

    nodes=(int)(params['nodes'])
    opt=optimizerL[params['optimizer']]
    opt=optimizerD[opt]
    activation=activationL[params['activation']]
    batch_size=(int)(params['batch_size'])
    epochs=(int)(params['epochs'])
    num_hidden_layers=(int)(params["num_hidden_layers"])
    #num_hidden_layers=3
    
    def get_model():    
        model=tf.keras.models.Sequential()
         
        model.add(tf.keras.layers.Dense(nodes,input_dim=X_train.shape[1],activation=activation))
        model.add(tf.keras.layers.Dropout(rate=params['dropout_prob']))             
            
        for i in range(num_hidden_layers-1):
            model.add(tf.keras.layers.Dense(nodes,activation=activation))
            model.add(tf.keras.layers.Dropout(rate=params['dropout_prob']))       
        
        model.add(tf.keras.layers.Dense(1,activation='sigmoid'))
        model.add(tf.keras.layers.Dropout(rate=params['dropout_prob']))
        
        model.compile(loss=tf.keras.losses.mean_squared_error,optimizer=opt,metrics=['mse'])
        return model

    es=tf.keras.callbacks.EarlyStopping(monitor='val_loss',mode='max',verbose=0,patience=20)
    nn=tf.keras.wrappers.scikit_learn.KerasRegressor(build_fn=get_model,epochs=epochs,batch_size=batch_size,verbose=0)
    
    
    kfold=KFold(n_splits=10,shuffle=True,random_state=random_state)
    score=-cross_val_score(nn,X_train,y_train,scoring='neg_mean_squared_error',cv=kfold,fit_params={'callbacks':[es]},n_jobs=-1).mean()
    
    return score

In [9]:
ann_trials=Trials()

best=fmin(fn=objective_ann,space=ann_params_space,algo=tpe.suggest,max_evals=500,rstate=np.random.default_rng(random_state),trials=ann_trials,verbose=True)


100%|██████████| 500/500 [57:41<00:00,  6.92s/trial, best loss: 3.2784476195009094e-05] 


In [13]:
d=pd.DataFrame(ann_trials.vals)

In [14]:
d

,activation,batch_size,dropout_prob,epochs,learning_rate,nodes,num_hidden_layers,optimizer
0,7,58.0,0.680782,200.0,0.004968,159.0,2.0,6
1,1,52.0,0.140312,230.0,0.020391,115.0,5.0,0
2,0,43.0,0.438650,250.0,0.015285,61.0,5.0,0
3,8,17.0,0.607171,410.0,0.002282,52.0,3.0,5
4,9,28.0,0.294136,300.0,0.003082,144.0,5.0,4
...,...,...,...,...,...,...,...,...
495,8,45.0,0.769796,420.0,0.074140,164.0,1.0,1
496,1,86.0,0.026214,450.0,0.044129,200.0,2.0,0
497,8,57.0,0.117513,340.0,0.101533,191.0,1.0,1
498,8,100.0,0.064435,350.0,0.197772,167.0,4.0,2


In [11]:
best

{'activation': 8,
 'batch_size': 97.0,
 'dropout_prob': 0.0009570121262778728,
 'epochs': 370.0,
 'learning_rate': 0.03939907301957611,
 'nodes': 115.0,
 'num_hidden_layers': 2.0,
 'optimizer': 1}

In [12]:
for i in range(0):
    print(i)

In [26]:
with open("dummy.csv","r") as f:
    data=f.read().replace('\t',',')
    
    print(data,file=open("dummy1.csv","w"))

In [30]:
cuo_water_data=pd.read_excel("H20_data.xlsx")
cuo_water_data

,Temperature,Nanparticle Heat Capacity,Volume fraction,CuO/water heat capacity
0,298.31,0.536,0.4,4.082
1,300.32,0.535,0.4,4.081
2,302.35,0.533,0.4,4.079
3,304.36,0.538,0.4,4.082
4,306.37,0.548,0.4,4.085
...,...,...,...,...
100,329.91,0.584,2.0,3.762
101,331.91,0.582,2.0,3.763
102,333.90,0.585,2.0,3.764
103,335.89,0.588,2.0,3.765


In [32]:
cuo_water_data.to_csv('Water_data.csv')